In [1]:
import pickle
import sys, os
sys.path.append("/Users/chilpert/Work/pyproteinsExt/src")
sys.path.append("/Users/chilpert/Work/pyproteins/src")
import pyproteinsExt 
import pyproteins
import time
from igraph import *

In [4]:
def save(data, tag=None):
    saveDir="/Volumes/arwen/mobi/group/NOX_CH/pickle_saved"
    timestr = time.strftime("%Y%m%d-%H%M%S")
    fTag = "NOX_annotation_" + tag + "_" if tag else "NOX_annotation_"
    fSerialDump = fTag + timestr + ".pickle"
    with open(saveDir + '/' + fSerialDump, 'wb') as f:
        pickle.dump(data, f)
    print('data structure saved to', saveDir + '/' + fSerialDump)

def load(fileName):
    saveDir="/Volumes/arwen/mobi/group/NOX_CH/pickle_saved"
    d = pickle.load( open(saveDir + "/" + fileName, "rb" ) )
    print("restore a annotated container of ", len(d), "elements")
    return d

In [5]:
data10_3=load("NOX_annotation_filter_fullPfam_filteredDomains1e-3_20190517-111109.pickle")
data10_1=load("NOX_annotation_filter_fullPfam_filteredDomains1e-1_20190517-111110.pickle")
data_all=load("NOX_annotation_filter_fullPfam_allDomains_20190517-111110.pickle")

restore a annotated container of  163 elements
restore a annotated container of  163 elements
restore a annotated container of  163 elements


In [6]:
dic_data={"1e-3":data10_3,"1e-1":data10_1,"all":data_all}

### Create graphs

In [9]:
def get_vertex_size(nb_domain,interval,min_size):
    if nb_domain==1: 
        return min_size 
    else: 
        size=min_size+interval*(nb_domain-1)
        return size
    
def get_edge_size(nb_occ,interval): 
    return nb_occ*interval

In [12]:
core_domains={"Ferric_reduct","FAD_binding_8","FAD_binding_6","NAD_binding_6","NAD_binding_1"}
for evalue in dic_data: 
    #Edge dictionnary 
    data=dic_data[evalue]
    dic_edges={}
    all_domains=set()
    for p in data: 
        domains=set(list(data[p]['hmmr'].keys()))
        for cd in core_domains: 
            domains.discard(cd)
        related_domains=domains.copy()
        for d in domains:
            related_domains.remove(d)
            all_domains.add(d)
            for d2 in related_domains : 
                edge=tuple(sorted((d,d2)))
                if edge not in dic_edges: 
                    dic_edges[edge]=0
                dic_edges[edge]+=1  
    
    dic_nb_domain={}
    for p in data : 
        for d in data[p]['hmmr']:
            if d not in core_domains: 
                if d not in dic_nb_domain :
                    dic_nb_domain[d]=0 
                dic_nb_domain[d]+=1
                
    list_edges=[]
    list_weight=[]
    for e in dic_edges: 
        list_edges.append(e)
        list_weight.append(dic_edges[e]) 
    
    g=Graph()
    g.add_vertices(len(all_domains))
    g.vs["name"]=list(all_domains)
    g.vs["label"]=g.vs["name"]
    g.add_edges(list_edges)
    for vertex in g.vs : 
        vertex["weight"]=dic_nb_domain[vertex["name"]]
        vertex["size"]=get_vertex_size(vertex["weight"],2,5)
    for e in g.es : 
        source=[v for v in g.vs if v.index==e.source][0] 
        target=[v for v in g.vs if v.index==e.target][0]
        edge_tuple=tuple(sorted((source["name"],target["name"])))
        nb_occ=dic_edges[edge_tuple]
        min_domains=min(source["weight"],target["weight"])
        e["weight"]=nb_occ/min_domains
        e["label"]=round(nb_occ/min_domains,2)
        e['width']=e["weight"]*5
        
    plot(g,vertex_label_dist=1.5,vertex_label_size=12,edge_label_dist=0,edge_color="grey",target="/Users/chilpert/Work/NOX_visu/graph_domains"+evalue+".pdf",margin=50)    
    
    connected_comp=g.clusters()
    
    nb_cc=0
    global_output="/Volumes/arwen/mobi/group/NOX_CH/Domain/domain"+evalue+"_components.tsv"
    go=open(global_output,"w")
    go.write("#Component\tDomains\tMean_weight\n")
    for cc in connected_comp.subgraphs(): 
        if len(cc.vs["name"])>1: 
            nb_cc+=1
            list_percent=[]
            print("== COMP",nb_cc)
            plot(cc,vertex_label_dist=1.5,vertex_label_size=25,edge_label_dist=0,edge_color="grey",target="/Users/chilpert/Work/NOX_visu/domains_"+evalue+"/comp"+str(nb_cc)+".pdf",margin=100,edge_label_size=20)
            mean_weight=mean(cc.es["weight"])
            go.write(str(nb_cc)+"\t"+",".join(cc.vs["name"])+"\t"+str(mean_weight)+"\n")
    go.close()      

== COMP 1
== COMP 1
== COMP 2
== COMP 3
== COMP 4
== COMP 5
== COMP 1
== COMP 2
== COMP 3
== COMP 4
== COMP 5
